# Insurance AI Assistant - Model Fine-tuning with LoRA/QLoRA

This notebook demonstrates fine-tuning large language models for insurance-specific tasks using LoRA (Low-Rank Adaptation) and QLoRA (Quantized LoRA) techniques.

## Objectives
- Load and configure base models (LLaMA, Mistral, Gemma)
- Implement QLoRA for memory-efficient fine-tuning
- Train the model on insurance datasets
- Monitor training progress and performance
- Save and evaluate the fine-tuned model

In [ ]:
# Install required packages for fine-tuning (uncomment if needed)
# !pip install transformers peft accelerate torch datasets wandb bitsandbytes

import os
import json
import torch
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Transformers and training libraries
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
from datasets import Dataset, load_dataset

# Optional: Weights & Biases for experiment tracking
try:
    import wandb
    WANDB_AVAILABLE = True
except ImportError:
    WANDB_AVAILABLE = False
    print("W&B not available - training will proceed without logging")

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f" Using device: {device}")
if torch.cuda.is_available():
    print(f" GPU: {torch.cuda.get_device_name()}")
    print(f" GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print(" Environment setup completed!")

## Model Configuration and Loading

Let's configure and load our base model with QLoRA settings for efficient fine-tuning.

In [ ]:
# Configuration
MODEL_NAME = "microsoft/DialoGPT-medium"  # Change to your preferred model
OUTPUT_DIR = "../models/insurance-assistant"
DATA_DIR = "../data/processed"

# Training hyperparameters
BATCH_SIZE = 4
LEARNING_RATE = 2e-4
NUM_EPOCHS = 3
MAX_LENGTH = 512

# LoRA configuration
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.1
TARGET_MODULES = ["q_proj", "v_proj"]  # Adjust based on model architecture

print(f" Model: {MODEL_NAME}")
print(f" Output directory: {OUTPUT_DIR}")
print(f" Training config: {NUM_EPOCHS} epochs, batch size {BATCH_SIZE}, LR {LEARNING_RATE}")
print(f" LoRA config: r={LORA_R}, alpha={LORA_ALPHA}, dropout={LORA_DROPOUT}")

# Ensure output directory exists
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
# QLoRA Configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print(" Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Add padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(" Added padding token")

print(" Loading model with QLoRA configuration...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

print(" Preparing model for k-bit training...")
model = prepare_model_for_kbit_training(model)

# LoRA Configuration
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

print(" Adding LoRA adapters...")
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()
print(" Model setup completed!")

## Dataset Loading and Preprocessing

Load and preprocess the insurance training data for fine-tuning.

In [ ]:
# Load datasets
def load_json_dataset(file_path):
    """Load dataset from JSON file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return Dataset.from_list(data)

# Load training and validation data
train_path = Path(DATA_DIR) / "train.json"
val_path = Path(DATA_DIR) / "validation.json"

if train_path.exists():
    train_dataset = load_json_dataset(train_path)
    print(f" Loaded {len(train_dataset)} training samples")
else:
    print(" Training data not found! Please run the data exploration notebook first.")

if val_path.exists():
    eval_dataset = load_json_dataset(val_path)
    print(f" Loaded {len(eval_dataset)} validation samples")
else:
    print("️ Validation data not found - will use training data split")
    split_dataset = train_dataset.train_test_split(test_size=0.1)
    train_dataset = split_dataset['train']
    eval_dataset = split_dataset['test']

# Preprocessing function
def preprocess_function(examples):
    """Preprocess data for training."""
    texts = []
    for i in range(len(examples['instruction'])):
        instruction = examples['instruction'][i]
        input_text = examples['input'][i]
        output_text = examples['output'][i]
        
        # Create Alpaca-style prompt
        if input_text.strip():
            prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output_text}"
        else:
            prompt = f"### Instruction:\n{instruction}\n\n### Response:\n{output_text}"
        
        texts.append(prompt)
    
    # Tokenize
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding=False,
        max_length=MAX_LENGTH,
        return_tensors=None,
    )
    
    # Set labels for causal LM
    tokenized["labels"] = tokenized["input_ids"].copy()
    
    return tokenized

# Preprocess datasets
print(" Preprocessing datasets...")
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

eval_dataset = eval_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=eval_dataset.column_names
)

print(f" Preprocessed {len(train_dataset)} training samples")
print(f" Preprocessed {len(eval_dataset)} validation samples")